In [1]:
import numpy as np
import global_vars_go as gvg

def switch_player_perspec(arry):
    arry = arry.reshape(gvg.board_size, gvg.board_size, gvg.board_channels)
    for i in range(len(arry)):
        for j in range(len(arry[i])):
            tmp = arry[i][j][gvg.player_channel]
            arry[i][j][gvg.player_channel] = arry[i][j][gvg.bot_channel]
            arry[i][j][gvg.bot_channel] = tmp
    return arry

def setup_board(game):
    preboard, plays = sgf_moves.get_setup_and_moves(game)
    rpreboard = preboard.board
    board = np.zeros((gvg.board_size, gvg.board_size, gvg.board_channels))
    if len(plays) < 1: # Return an empty board if the game has no moves
        return board
    if plays[0][0] == "b":
        color_stone = gvg.bot_channel
    else:
        color_stone = gvg.player_channel
    for i in range(len(rpreboard)):
        for j in range(len(rpreboard[i])):
            if rpreboard[i][j] == "b":
                board[i][j][color_stone] = gvg.filled

    return board

In [2]:
import glob
import os
import sys
from sgfmill.sgfmill import sgf, sgf_moves
import numpy
import board3d as go_board

num_games = 2000
kifuPath = "../kifu/"

print("Loading game data...")

X = []
Y = []

i = 0
for filename in glob.glob(os.path.join(kifuPath, "*.sgf")):
    if i < num_games:
        with open(filename, "rb") as f:
            game = sgf.Sgf_game.from_bytes(f.read())
            board = setup_board(game)
            for node in game.get_main_sequence():
                board = switch_player_perspec(board)
                node_move = node.get_move()[1]
                if node_move is not None:
                    X.append(np.copy(board))
                    move_board = np.zeros((19, 19))
                    move_board[node_move[0]][node_move[1]] = 1
                    Y.append(move_board)
                    board = go_board.make_move(board, node_move, gvg.bot_channel, gvg.player_channel, True)
    i += 1

X = np.array(X).reshape(-1, 19, 19, 2)
Y = np.array(Y).reshape(-1, 19 * 19)

print("Done loading {} board states".format(len(X)))

Loading game data...
Done loading 397849 board states


In [ ]:
import tensorflow as tf

dev = 0.02

x = tf.placeholder(tf.float32, shape=(None, 19, 19, 2))
y = tf.placeholder(tf.float32, shape=(None, 19*19))

keep_prob = tf.placeholder(tf.float32)

#TODO: Figure out about how exactly weights are created (uniform)
wc1 = tf.get_variable("wc1", shape=[3, 3, 2, 32], dtype=tf.float32, 
                      initializer=tf.random_normal_initializer(stddev=dev))
bc1 = tf.get_variable("bc1", 32, dtype=tf.float32,
                    initializer=tf.constant_initializer(0.))

wc2 = tf.get_variable("wc2", shape=[3, 3, 32, 64], dtype=tf.float32, 
                      initializer=tf.random_normal_initializer(stddev=dev))
bc2 = tf.get_variable("bc2", 64, dtype=tf.float32,
                    initializer=tf.constant_initializer(0.))

wfc1 = tf.get_variable("wfc1", shape=[19*19*64, 128], dtype=tf.float32, 
                      initializer=tf.truncated_normal_initializer(stddev=dev))
bfc1 = tf.get_variable("bfc1", 128, dtype=tf.float32,
                    initializer=tf.constant_initializer(0.))

wfc2 = tf.get_variable("wfc2", shape=[128, 256], dtype=tf.float32, 
                      initializer=tf.truncated_normal_initializer(stddev=dev))
bfc2 = tf.get_variable("bfc2", 256, dtype=tf.float32,
                    initializer=tf.constant_initializer(0.))

wfc3 = tf.get_variable("wfc3", shape=[256, 19*19], dtype=tf.float32, 
                      initializer=tf.truncated_normal_initializer(stddev=dev))
bfc3 = tf.get_variable("bfc3", 19*19, dtype=tf.float32,
                    initializer=tf.constant_initializer(0.))

x = tf.reshape(x, [-1, 19, 19, 2])

l1 = tf.nn.conv2d(x, wc1, [1, 1, 1, 1], "SAME")
l1 = tf.nn.bias_add(l1, bc1)
l1 = tf.nn.relu(l1)

l2 = tf.nn.conv2d(l1, wc2, [1, 1, 1, 1], "SAME")
l2 = tf.nn.bias_add(l2, bc2)
l2 = tf.nn.relu(l2)

l2 = tf.reshape(l2, [-1, 19*19*64])

l3 = tf.matmul(l2, wfc1)
l3 = tf.nn.bias_add(l3, bfc1)
l3 = tf.nn.tanh(l3)

l3_drop = tf.nn.dropout(l3, keep_prob)

l4 = tf.matmul(l3_drop, wfc2)
l4 = tf.nn.bias_add(l4, bfc2)
l4 = tf.nn.tanh(l4)

l4_drop = tf.nn.dropout(l4, keep_prob)

output = tf.matmul(l4_drop, wfc3)
output = tf.nn.bias_add(output, bfc3)
output = tf.nn.softmax(output)

optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

# Implementation from tflearn
_EPSILON = 1e-10
_FLOATX = tf.float32
output /= tf.reduce_sum(output, reduction_indices=len(output.get_shape())-1, keep_dims=True)
output = tf.clip_by_value(output, tf.cast(_EPSILON, dtype=_FLOATX), tf.cast(1.-_EPSILON, dtype=_FLOATX))
cross_entropy = - tf.reduce_sum(y * tf.log(output), reduction_indices=len(output.get_shape())-1)

loss = tf.reduce_mean(cross_entropy)

optimizer = optimizer.minimize(loss)

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(output, 1), tf.argmax(y, 1)), tf.float32))

In [ ]:
from IPython.display import clear_output

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    hm_epochs = 50
    
    for epoch in range(hm_epochs):
        tl = 0
        ta = 0

        for i in range(int(len(X)/150)):
            _, l = sess.run([optimizer, loss], feed_dict={x: X[i*150:i*150+125], y: Y[i*150:i*150+125], keep_prob: 0.65})
            a = sess.run(accuracy, feed_dict={x: X[i*150+125:i*150+150], y: Y[i*150+125:i*150+150], keep_prob: 1.0})
            tl += l
            ta += a

            if i % 8 == 0:
                ta /= 8
                tl /= 8
                clear_output(True)
                print("Epoch: {} | Loss: {} | Acc: {}".format(epoch+1, round(tl, 4), round(ta, 4)))
                tl = 0
                ta = 0

Epoch: 7 | Loss: 4.8869 | Acc: 0.09
